In [1]:
#library import

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from collections import Counter
from scipy.stats import entropy
import math
from functools import reduce
from sklearn.feature_selection import mutual_info_classif


In [2]:
#data reading

data = pd.read_csv('parkinson.csv')
data.head()

,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,...,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,...,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,...,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,...,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,1,0,0.32790,0.79782,0.53028,236,235,0.008162,0.002669,0.00535,...,6.1727,5.8416,6.0805,5.7621,7.7817,11.6891,8.2103,5.0559,6.1164,1


In [4]:
data.index=range(0,len(data),1)
data.head()

,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,...,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,...,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,...,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,...,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,...,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,1,0,0.32790,0.79782,0.53028,236,235,0.008162,0.002669,0.00535,...,6.1727,5.8416,6.0805,5.7621,7.7817,11.6891,8.2103,5.0559,6.1164,1


In [5]:
data.dropna()
data.isnull().sum()

id                           0
gender                       0
PPE                          0
DFA                          0
RPDE                         0
                            ..
tqwt_kurtosisValue_dec_33    0
tqwt_kurtosisValue_dec_34    0
tqwt_kurtosisValue_dec_35    0
tqwt_kurtosisValue_dec_36    0
class                        0
Length: 755, dtype: int64

In [6]:
#data split

X = data.drop(['class'], axis=1)
y = data['class']
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.6)
print(len(x_train),len(x_test),len(y_train),len(y_test))

302 454 302 454


In [7]:
#accuracies with classifier run WITHOUT feature selection

model1=SVC()
model2=GaussianNB()
model3 = DecisionTreeClassifier()
model4 = KNeighborsClassifier()
model1.fit(x_train, y_train)
print ('SVC Score:' ,model1.score(x_test, y_test))
model2.fit(x_train, y_train)
print ('NB Score:' ,model2.score(x_test, y_test))
model3.fit(x_train, y_train)
print ('DT Score:' ,model3.score(x_test, y_test))
model4.fit(x_train, y_train)
print ('KNN Score:' ,model4.score(x_test, y_test))

SVC Score: 0.73568281938326
NB Score: 0.7268722466960352
DT Score: 0.7577092511013216
KNN Score: 0.7224669603524229


In [8]:
#FILTER TECHNIQUE 1

#correlation

df1 = X
op=y
a1=[]
c1=[]
for i in df1.columns.values:
        df1[i]=df1[i].astype('float64')
        correlation=op.corr(df1[i])
        if correlation>0.2:
            a1.append(correlation)
            c1.append(i)

dfc1=pd.DataFrame(c1)
dfa1=pd.DataFrame(a1)
info1 = pd.concat([dfc1,dfa1], axis=1)
info1.columns = ['name','correlation']
info1

,name,correlation
0,DFA,0.306070
1,RPDE,0.247444
2,meanPeriodPulses,0.211368
3,locPctJitter,0.204228
4,apq11Shimmer,0.227427
...,...,...
69,tqwt_minValue_dec_16,0.244959
70,tqwt_kurtosisValue_dec_18,0.208645
71,tqwt_kurtosisValue_dec_20,0.225739
72,tqwt_kurtosisValue_dec_35,0.219454


In [9]:
#selecting top 50 features

top1 = info1.nlargest(50,'correlation')
SelectedFeatures1= np.array(top1['name'])
SelectedFeatures1

array(['mean_MFCC_2nd_coef', 'tqwt_minValue_dec_12',
       'tqwt_minValue_dec_11', 'tqwt_minValue_dec_13',
       'std_9th_delta_delta', 'std_8th_delta_delta',
       'std_7th_delta_delta', 'std_6th_delta_delta', 'std_8th_delta',
       'std_10th_delta_delta', 'std_9th_delta', 'std_11th_delta_delta',
       'std_6th_delta', 'std_7th_delta', 'std_delta_delta_log_energy',
       'tqwt_minValue_dec_14', 'std_10th_delta', 'std_11th_delta', 'DFA',
       'std_12th_delta_delta', 'tqwt_minValue_dec_10',
       'app_entropy_shannon_10_coef', 'app_entropy_shannon_9_coef',
       'app_entropy_shannon_8_coef', 'app_entropy_shannon_7_coef',
       'app_entropy_shannon_6_coef', 'app_entropy_shannon_5_coef',
       'app_entropy_shannon_4_coef', 'app_entropy_shannon_3_coef',
       'app_entropy_shannon_2_coef', 'app_entropy_shannon_1_coef',
       'std_12th_delta', 'tqwt_kurtosisValue_dec_36',
       'tqwt_entropy_log_dec_26', 'std_4th_delta_delta', 'std_4th_delta',
       'tqwt_minValue_dec_15', 't

In [10]:
#FILTER TECHNIQUE 2

#info gain ratio

from scipy.stats import entropy
def info_gain(Ex,a):
    H_Ex = entropy(list(Counter(Ex).values()))
    # Compute the sum of all values v in a
    sum_v = 0
    for v in set(a):
        Ex_a_v = [x for x, t in zip(Ex, a) if t == v]
        sum_v += (len(Ex_a_v) / len(Ex)) *\
                 (entropy(list(Counter(Ex_a_v).values())))

    # Return result
    return H_Ex - sum_v


def intrinsic_value(Ex, a):
    # Compute the sum of all values v in a
    sum_v = 0
    for v in set(a):
        Ex_a_v = [x for x, t in zip(Ex, a) if t == v]
        sum_v += (len(Ex_a_v) / len(Ex)) * math.log(len(Ex_a_v) / len(Ex), 2)

    # Return result
    return -sum_v


def info_gain_ratio(Ex, a):
        # Check whether examples and attributes have the same lengths.
    if len(Ex) != len(a):
        raise ValueError("Ex and a must be of the same size.")

    # Compute information gain ratio as IG/IV
    return info_gain(Ex, a) / intrinsic_value(Ex, a)


In [11]:
ar=[]
br=[]
iv=[]
ig=[]
for (columnName, columnData) in X.iteritems():
        a=info_gain_ratio(columnData, y)
        c=intrinsic_value(columnData, y)
        d=info_gain(columnData, y)
        ar.append(a)
        br.append(columnName)
        iv.append(c)
        ig.append(d)
a1 = pd.DataFrame(ar)
b1 = pd.DataFrame(br)
info2 = pd.concat([a1,b1], axis=1)
info2.columns = ['Score','Features']
info2

,Score,Features
0,0.693147,id
1,0.020593,gender
2,0.681085,PPE
3,0.686418,DFA
4,0.688661,RPDE
...,...,...
749,0.686418,tqwt_kurtosisValue_dec_32
750,0.688661,tqwt_kurtosisValue_dec_33
751,0.693147,tqwt_kurtosisValue_dec_34
752,0.690904,tqwt_kurtosisValue_dec_35


In [12]:
#selecting top 50 features

top2 = info2.nlargest(50,'Score')
SelectedFeatures2= np.array(top2['Features'])
SelectedFeatures2

array(['id', 'b4', 'mean_MFCC_8th_coef', 'mean_0th_delta',
       'mean_1st_delta', 'mean_11th_delta', 'std_6th_delta',
       'det_entropy_log_6_coef', 'det_TKEO_mean_5_coef',
       'det_TKEO_std_3_coef', 'app_entropy_log_4_coef',
       'app_entropy_log_10_coef', 'det_LT_entropy_shannon_1_coef',
       'det_LT_entropy_log_9_coef', 'det_LT_TKEO_std_8_coef',
       'app_LT_TKEO_mean_4_coef', 'app_LT_TKEO_std_4_coef',
       'tqwt_energy_dec_9', 'tqwt_energy_dec_10', 'tqwt_energy_dec_14',
       'tqwt_energy_dec_20', 'tqwt_energy_dec_21',
       'tqwt_entropy_shannon_dec_4', 'tqwt_entropy_shannon_dec_13',
       'tqwt_TKEO_mean_dec_14', 'tqwt_TKEO_mean_dec_16',
       'tqwt_TKEO_mean_dec_25', 'tqwt_TKEO_std_dec_26',
       'tqwt_stdValue_dec_20', 'tqwt_stdValue_dec_30',
       'tqwt_stdValue_dec_34', 'tqwt_stdValue_dec_35',
       'tqwt_minValue_dec_5', 'tqwt_minValue_dec_7',
       'tqwt_minValue_dec_13', 'tqwt_maxValue_dec_20',
       'tqwt_maxValue_dec_25', 'tqwt_maxValue_dec_26',
 

In [13]:
#FILTER TECHNIQUE 3

#Symmetrical Uncertainty


#entropy
from scipy.stats import entropy

#Joint Entropy
def jEntropy(Y,X):
    YX = np.c_[Y,X]
    return entropy(YX)

#conditional entropy
def cEntropy(Y, X):
    return jEntropy(Y, X) - entropy(X)

#SU
def symmetricalUncertain(Y,X):
    n = float(y.shape[0])
    vals = np.unique(Y)
    # Computing Entropy for the feature x. 
    Hx = entropy(X)
    # Computing Entropy for the feature y.
    Hy = entropy(Y)
    #Computing Joint entropy between x and y.
    Hxy = jEntropy(Y,X)
    IG = Hx-Hxy
    return 2*IG/(Hx+Hy)

In [14]:
top_n1 =[]
for i in X.columns.values:
    X[i]=X[i].astype('float64')
    top_n1.append(symmetricalUncertain(X[i],y))
#print (top_n1)
top_n2=[row[0] for row in top_n1]
print (top_n2)

col_name1 = np.array(X.columns)
a1 = pd.DataFrame(top_n2)
b1 = pd.DataFrame(col_name1)
info3 = pd.concat([a1,b1], axis=1)
info3.columns = ['Score','Features']
info3


<ipython-input-13-677bcc6d6073>:29: RuntimeWarning: invalid value encountered in true_divide
  return 2*IG/(Hx+Hy)


[-0.01532484923111689, 0.059979104911067484, -0.04014998094629275, -0.04445373792218621, -0.03914002732010321, -0.03816739285632279, -0.03806178514838645, -0.03915173233762445, 0.13732784143556723, 0.014898541265130655, 0.03931297553885873, 0.0616236247540486, 0.040880486377066975, 0.06158199411974554, -0.017738642246769746, -0.01770479315307786, -0.016189537819829902, -0.016374705130363324, -0.020156992668447463, -0.016189579057487424, -0.044838620601469314, 0.12117718546562693, -0.037609008955861854, -0.0439048816987751, -0.04463188590025568, -0.04448092654063084, -0.0418451772790125, -0.04380867587112991, -0.04333562378385711, -0.044474115145133906, 0.025708735294261723, 0.04704589664467318, 0.05007880748073965, 0.03281146803891219, -0.04281684856819879, 0.015156454002176283, 0.12866624643398628, -0.038931430830191356, -0.021759153204819227, -0.03702970954658219, -0.041545023738912265, -0.04508648026508969, -0.04518452827606753, 0.049927476150886316, 0.052678621985279984, 0.05873398

,Score,Features
0,-0.015325,id
1,0.059979,gender
2,-0.040150,PPE
3,-0.044454,DFA
4,-0.039140,RPDE
...,...,...
749,0.065156,tqwt_kurtosisValue_dec_32
750,0.051447,tqwt_kurtosisValue_dec_33
751,0.028529,tqwt_kurtosisValue_dec_34
752,0.020831,tqwt_kurtosisValue_dec_35


In [15]:
#selecting top 50

top3 = info3.nlargest(50,'Score')
SelectedFeatures3 = np.array(top3['Features'])
SelectedFeatures3

array(['tqwt_TKEO_mean_dec_29', 'Ed_1_coef', 'tqwt_TKEO_mean_dec_33',
       'Ed_2_coef', 'tqwt_energy_dec_33', 'tqwt_TKEO_mean_dec_32',
       'det_TKEO_mean_1_coef', 'tqwt_TKEO_mean_dec_30',
       'tqwt_energy_dec_34', 'Ed_3_coef', 'det_TKEO_std_1_coef',
       'det_TKEO_std_8_coef', 'tqwt_TKEO_std_dec_33',
       'det_TKEO_mean_8_coef', 'det_TKEO_mean_2_coef',
       'det_TKEO_mean_3_coef', 'tqwt_TKEO_std_dec_29',
       'tqwt_TKEO_std_dec_32', 'Ed2_1_coef', 'tqwt_energy_dec_35',
       'det_LT_TKEO_std_10_coef', 'det_TKEO_std_7_coef',
       'det_LT_TKEO_mean_1_coef', 'det_LT_TKEO_mean_10_coef',
       'det_TKEO_std_2_coef', 'det_LT_TKEO_mean_8_coef',
       'det_LT_TKEO_std_8_coef', 'Ed2_10_coef', 'Ed_5_coef',
       'tqwt_energy_dec_29', 'det_TKEO_std_10_coef', 'Ed_8_coef',
       'det_TKEO_mean_7_coef', 'Ed_9_coef', 'det_TKEO_mean_10_coef',
       'Ed2_8_coef', 'tqwt_energy_dec_32', 'det_LT_TKEO_std_7_coef',
       'Ed_10_coef', 'Ed2_2_coef', 'Ed_4_coef', 'Ed_7_coef',
       'd

In [16]:
#Selecting 5k subset

def top_fs_union(n):
    f1 = SelectedFeatures1[:n]
    f2 = SelectedFeatures2[:n]
    f3 = SelectedFeatures3[:n]
    f_3k = reduce(np.union1d, (f1,f2,f3))
    f_2k = ((len(f_3k)*2)//3)
    f_2k_random = np.random.choice(f_3k,f_2k,replace=False)
    f_5k = reduce(np.union1d, (f_3k,f_2k_random))
    #print (f_5k)
    return(f_5k)

In [17]:
#Assemble Function RUN

def assemble_run(n,classifier,x_train, x_test, y_train, y_test):
  top_fs=top_fs_union(n)
  top_fs=np.append(top_fs,['class'], axis=0)
  df = data.loc[:,top_fs]
  X = df.drop(['class'], axis=1)
  y = df['class']
  model =classifier()#modify the code ,keep classifier in different function and call here
  model.fit(x_train, y_train)
  print (model.score(x_test,y_test))

In [18]:
#accuracies with classifier run WITH feature selection

for i in (5,10,15,20,25,30,40,50):
    print('Score for SVM by Union of '+str(i)+' Best Features:',end=' ')
    feature_run=assemble_run(i,SVC,x_train, x_test, y_train, y_test)
print('\n\n')

#NaiveBayes
for i in (5,10,15,20,25,30,40,50):
    print('Score for Naive Bayes by Union of '+str(i)+' Best Features:',end=' ')
    feature_run=assemble_run(i,GaussianNB,x_train, x_test, y_train, y_test)
print('\n\n')

#DecisionTree
for i in (5,10,15,20,25,30,40,50):
    print('Score for DecisionTree by Union of '+str(i)+' Best Features:',end=' ')
    feature_run=assemble_run(i,DecisionTreeClassifier,x_train, x_test, y_train, y_test)
print('\n\n')

#KNN
for i in (5,10,15,20,25,30,40,50):
    print('Score for KNN by Union of '+str(i)+' Best Features:',end=' ')
    feature_run=assemble_run(i,KNeighborsClassifier,x_train, x_test, y_train, y_test)
print('\n\n')


Score for SVM by Union of 5 Best Features: 0.73568281938326
Score for SVM by Union of 10 Best Features: 0.73568281938326
Score for SVM by Union of 15 Best Features: 0.73568281938326
Score for SVM by Union of 20 Best Features: 0.73568281938326
Score for SVM by Union of 25 Best Features: 0.73568281938326
Score for SVM by Union of 30 Best Features: 0.73568281938326
Score for SVM by Union of 40 Best Features: 0.73568281938326
Score for SVM by Union of 50 Best Features: 0.73568281938326



Score for Naive Bayes by Union of 5 Best Features: 0.7268722466960352
Score for Naive Bayes by Union of 10 Best Features: 0.7268722466960352
Score for Naive Bayes by Union of 15 Best Features: 0.7268722466960352
Score for Naive Bayes by Union of 20 Best Features: 0.7268722466960352
Score for Naive Bayes by Union of 25 Best Features: 0.7268722466960352
Score for Naive Bayes by Union of 30 Best Features: 0.7268722466960352
Score for Naive Bayes by Union of 40 Best Features: 0.7268722466960352
Score for Naiv